# Calculation of Percentiles by industry and feature - Quarterly period

I will calculate percetiles on the industry firms for two variables to start: 'eff_tax_rate%' and 'deduc_to_income_ratio'.

I will calculate an overall percentile, and a industry percentile

In [36]:
%load_ext autoreload

In [37]:
%autoreload 2

In [38]:
import pandas as pd
import numpy as np

from dstools.mlutils.corp_tax_audit_unsupervised import (
                    add_percentile,
                    add_abs_diffs,
                    get_kmean_clusters,
                    get_inertia_vs_k,
                    get_cluster_counts,
                    get_outlier_scores
)
import hvplot.pandas
import matplotlib.pyplot as plt
from typing import List, Dict

In [4]:
%store -r q_industries_original
q_industries_original.head()

,naics_code,business_id,bus_loc_id,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,sumsum_deduc,sumsum_taxable,sumsum_paid,eff_tax_rate%,deduc_to_income_ratio,naics2
date,,,,,,,,,,,,,,
2001Q1,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,16193.58,0.00,16193.58,67.20,0.414979,0.000000,22
2001Q2,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,158968.92,0.00,158968.92,659.72,0.414999,0.000000,22
2001Q3,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,95335.05,20497.55,74837.50,310.58,0.325777,0.215005,22
2001Q4,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,36138.27,36138.27,0.00,0.00,0.000000,1.000000,22
2002Q1,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,7334.92,0.00,7334.92,30.44,0.415001,0.000000,22


In [193]:
#q_industries = q_industries_original
if type(q_industries.index)== pd.core.indexes.range.RangeIndex:
    q_industries.set_index('date', inplace=True, drop=True)
print(type(q_industries.index))
print(q_industries.columns)

<class 'pandas.core.indexes.period.PeriodIndex'>
Index(['index', 'naics2', 'business_id', 'bus_loc_id', 'naics_code',
       'business_legal_name', 'maxNumLoc', 'tax_period_cd', 'obl_type_id',
       'sumsum_gross', 'sumsum_deduc', 'sumsum_taxable', 'sumsum_paid',
       'eff_tax_rate%', 'deduc_to_income_ratio', 'eff_tax%_perc_glob',
       'eff_tax%_perc_ind', 'deduc2income_perc_glob', 'deduc2income_perc_ind',
       '4Delta_abs_eff_tax_rate%', '4Delta_abs_deduc_to_income_ratio',
       '4D_eff_tax%_perc_glob', '4D_deduc2income_perc_glob',
       '4D_eff_tax_perc_ind', '4D_deduc2income_perc_ind', 'k-cluster_ind',
       'k-cluster_g', 'sumsum_gross_perc_glob', 'sumsum_gross_perc_ind'],
      dtype='object')


In [6]:
# Adding Global effective tax rate percentile
feature = 'eff_tax_rate%'
new_name = 'eff_tax%_perc_glob'
add_percentile(df=q_industries, col=feature, new_name=new_name)
q_industries.head()

,naics_code,business_id,bus_loc_id,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,sumsum_deduc,sumsum_taxable,sumsum_paid,eff_tax_rate%,deduc_to_income_ratio,naics2,eff_tax%_perc_glob
date,,,,,,,,,,,,,,,
2001Q1,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,16193.58,0.00,16193.58,67.20,0.414979,0.000000,22,0.721733
2001Q2,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,158968.92,0.00,158968.92,659.72,0.414999,0.000000,22,0.814791
2001Q3,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,95335.05,20497.55,74837.50,310.58,0.325777,0.215005,22,0.625852
2001Q4,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,36138.27,36138.27,0.00,0.00,0.000000,1.000000,22,0.003649
2002Q1,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,7334.92,0.00,7334.92,30.44,0.415001,0.000000,22,0.906874


In [7]:
# Add single industry percentile - Effecctive Tax Rate %
# IMPORTANT: columns of the q_industries may not have to contain the percentiles
feature = 'eff_tax_rate%'
new_name = 'eff_tax%_perc_ind'


for naics2_code in q_industries['naics2'].unique():
    # Create a subset DataFrame for the current naics2 code
    single_industry_df = q_industries[q_industries['naics2'] == naics2_code].copy()
    
    # Calculate the percentiles within the group
    # (Obs: When applying the function, the assingment using loc did not work, so reconstruct calculation.)
    single_industry_df[new_name] = single_industry_df[[feature]].groupby(
                                            by=[single_industry_df.index.year, 
                                                single_industry_df.index.quarter
                                               ]).rank(method="average", pct=True)
 
    # Update the original DataFrame with the calculated percentiles
    q_industries.loc[q_industries['naics2'] == naics2_code, new_name] = single_industry_df[new_name]
    #print(q_industries[new_name].isnull().sum())

q_industries.head()


,naics_code,business_id,bus_loc_id,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,sumsum_deduc,sumsum_taxable,sumsum_paid,eff_tax_rate%,deduc_to_income_ratio,naics2,eff_tax%_perc_glob,eff_tax%_perc_ind
date,,,,,,,,,,,,,,,,
2001Q1,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,16193.58,0.00,16193.58,67.20,0.414979,0.000000,22,0.721733,0.909091
2001Q2,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,158968.92,0.00,158968.92,659.72,0.414999,0.000000,22,0.814791,1.000000
2001Q3,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,95335.05,20497.55,74837.50,310.58,0.325777,0.215005,22,0.625852,0.909091
2001Q4,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,36138.27,36138.27,0.00,0.00,0.000000,1.000000,22,0.003649,0.090909
2002Q1,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,7334.92,0.00,7334.92,30.44,0.415001,0.000000,22,0.906874,1.000000


In [8]:
q_industries['eff_tax%_perc_glob'].describe()

count    847245.000000
mean          0.500060
std           0.288677
min           0.000922
25%           0.250053
50%           0.500053
75%           0.750081
max           1.000000
Name: eff_tax%_perc_glob, dtype: float64

In [9]:
# Adding Global deduction to income ratio
feature = 'deduc_to_income_ratio'
new_name = 'deduc2income_perc_glob'
add_percentile(df=q_industries, col=feature, new_name=new_name)
q_industries.head()

,naics_code,business_id,bus_loc_id,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,sumsum_deduc,sumsum_taxable,sumsum_paid,eff_tax_rate%,deduc_to_income_ratio,naics2,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob
date,,,,,,,,,,,,,,,,,
2001Q1,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,16193.58,0.00,16193.58,67.20,0.414979,0.000000,22,0.721733,0.909091,0.390064
2001Q2,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,158968.92,0.00,158968.92,659.72,0.414999,0.000000,22,0.814791,1.000000,0.389619
2001Q3,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,95335.05,20497.55,74837.50,310.58,0.325777,0.215005,22,0.625852,0.909091,0.912581
2001Q4,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,36138.27,36138.27,0.00,0.00,0.000000,1.000000,22,0.003649,0.090909,0.997867
2002Q1,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,7334.92,0.00,7334.92,30.44,0.415001,0.000000,22,0.906874,1.000000,0.389933


In [10]:
# Notice that up to percentile 80 still values are zero for the deduc to income ratio
digits = 3
value_counts = q_industries['deduc_to_income_ratio'].round(digits).value_counts()
percentage = 100 * value_counts / q_industries.shape[0]

result = pd.concat([value_counts, percentage], axis=1)
result.columns = ['Value Counts for deduc to income ratio', 'Frequency %']
result

,Value Counts for deduc to income ratio,Frequency %
0.000,681093,80.389144
0.001,3509,0.414166
0.002,2929,0.345709
0.003,2404,0.283743
0.004,2102,0.248098
...,...,...
0.588,60,0.007082
0.682,59,0.006964
0.730,59,0.006964
0.715,57,0.006728


In [11]:
# Add single industry percentile - deduc to income ratio
# IMPORTANT: columns of the q_industries may not have to contain the percentiles
feature = 'deduc_to_income_ratio'
new_name = 'deduc2income_perc_ind'
    
for naics2_code in q_industries['naics2'].unique():
    # Create a subset DataFrame for the current naics2 code
    single_industry_df = q_industries[q_industries['naics2'] == naics2_code].copy()
    
    # Calculate the percentiles (Obs: When applying the function, the assingment using loc where not working)
    single_industry_df[new_name] = single_industry_df[[feature]].groupby(
                                            by=[single_industry_df.index.year, 
                                                single_industry_df.index.quarter
                                               ]).rank(method="average", pct=True)
 
    # Update the original DataFrame with the calculated percentiles
    q_industries.loc[q_industries['naics2'] == naics2_code, new_name] = single_industry_df[new_name]
    #print(q_industries[new_name].isnull().sum())

q_industries.head()

,naics_code,business_id,bus_loc_id,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,sumsum_deduc,sumsum_taxable,sumsum_paid,eff_tax_rate%,deduc_to_income_ratio,naics2,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind
date,,,,,,,,,,,,,,,,,,
2001Q1,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,16193.58,0.00,16193.58,67.20,0.414979,0.000000,22,0.721733,0.909091,0.390064,0.454545
2001Q2,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,158968.92,0.00,158968.92,659.72,0.414999,0.000000,22,0.814791,1.000000,0.389619,0.454545
2001Q3,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,95335.05,20497.55,74837.50,310.58,0.325777,0.215005,22,0.625852,0.909091,0.912581,0.909091
2001Q4,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,36138.27,36138.27,0.00,0.00,0.000000,1.000000,22,0.003649,0.090909,0.997867,1.000000
2002Q1,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,7334.92,0.00,7334.92,30.44,0.415001,0.000000,22,0.906874,1.000000,0.389933,0.454545


%store q_industries

In [12]:
# delete?
tsample = q_industries[(q_industries.index.year == 2010) #& (q_industries.index.quarter == 2)
                      &(q_industries['naics2']=='23')][['eff_tax_rate%', 'eff_tax%_perc_ind']]
tsample.sort_values(by='eff_tax%_perc_ind').hvplot(kind='scatter', y='eff_tax%_perc_ind', x='eff_tax_rate%')

:Scatter   [eff_tax_rate%]   (eff_tax%_perc_ind)

In [13]:
#delete?
tsample = tsample.sort_values(by='eff_tax_rate%')
tsample[(tsample['eff_tax_rate%']>0.4080)&(tsample['eff_tax_rate%']<0.41)]

,eff_tax_rate%,eff_tax%_perc_ind
date,,
2010Q3,0.408266,0.910377
2010Q4,0.408513,0.909842
2010Q2,0.408558,0.901143
2010Q2,0.408985,0.901816
2010Q1,0.408997,0.897315
2010Q1,0.409619,0.897987
2010Q2,0.409955,0.902488


In [14]:
tsample = tsample.sort_values(by='eff_tax_rate%')
tsample[(tsample['eff_tax%_perc_ind']>0.2495)&(tsample['eff_tax%_perc_ind']<0.2505)]

,eff_tax_rate%,eff_tax%_perc_ind
date,,
2010Q4,0.214996,0.249828
2010Q1,0.214996,0.250000
2010Q1,0.214996,0.250000
2010Q3,0.214997,0.250000
2010Q2,0.214997,0.250168


In [15]:
q_industries.head()

,naics_code,business_id,bus_loc_id,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,sumsum_deduc,sumsum_taxable,sumsum_paid,eff_tax_rate%,deduc_to_income_ratio,naics2,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind
date,,,,,,,,,,,,,,,,,,
2001Q1,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,16193.58,0.00,16193.58,67.20,0.414979,0.000000,22,0.721733,0.909091,0.390064,0.454545
2001Q2,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,158968.92,0.00,158968.92,659.72,0.414999,0.000000,22,0.814791,1.000000,0.389619,0.454545
2001Q3,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,95335.05,20497.55,74837.50,310.58,0.325777,0.215005,22,0.625852,0.909091,0.912581,0.909091
2001Q4,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,36138.27,36138.27,0.00,0.00,0.000000,1.000000,22,0.003649,0.090909,0.997867,1.000000
2002Q1,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,7334.92,0.00,7334.92,30.44,0.415001,0.000000,22,0.906874,1.000000,0.389933,0.454545


## Percentiles on the differences

In [16]:
q_industries = add_abs_diffs(X=q_industries, 
                    abs_diff_cols= ['eff_tax_rate%', 'deduc_to_income_ratio'], 
                    reference_cols=['naics2', 'business_id', 'bus_loc_id']
                  ) 

['naics2', 'business_id', 'bus_loc_id', 'date']


In [17]:
q_industries.columns

Index(['naics2', 'business_id', 'bus_loc_id', 'naics_code',
       'business_legal_name', 'maxNumLoc', 'tax_period_cd', 'obl_type_id',
       'sumsum_gross', 'sumsum_deduc', 'sumsum_taxable', 'sumsum_paid',
       'eff_tax_rate%', 'deduc_to_income_ratio', 'eff_tax%_perc_glob',
       'eff_tax%_perc_ind', 'deduc2income_perc_glob', 'deduc2income_perc_ind',
       '1Delta_abs_eff_tax_rate%', '1Delta_abs_deduc_to_income_ratio',
       '4Delta_abs_eff_tax_rate%', '4Delta_abs_deduc_to_income_ratio'],
      dtype='object')

In [18]:
# Adding Global effective tax rate percentile
feature = '4Delta_abs_eff_tax_rate%'
new_name = '4D_eff_tax%_perc_glob'
add_percentile(df=q_industries, col=feature, new_name=new_name)
q_industries.head()

,naics2,business_id,bus_loc_id,naics_code,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,sumsum_deduc,...,deduc_to_income_ratio,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,1Delta_abs_eff_tax_rate%,1Delta_abs_deduc_to_income_ratio,4Delta_abs_eff_tax_rate%,4Delta_abs_deduc_to_income_ratio,4D_eff_tax%_perc_glob
date,,,,,,,,,,,,,,,,,,,,,
2001Q1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,16193.58,0.00,...,0.000000,0.721733,0.909091,0.390064,0.454545,0.000000,0.000000,0.000000,0.0,0.500045
2001Q2,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,158968.92,0.00,...,0.000000,0.814791,1.000000,0.389619,0.454545,0.000020,0.000000,0.000000,0.0,0.500184
2001Q3,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,95335.05,20497.55,...,0.215005,0.625852,0.909091,0.912581,0.909091,-0.089222,0.215005,0.000000,0.0,0.500093
2001Q4,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,36138.27,36138.27,...,1.000000,0.003649,0.090909,0.997867,1.000000,-0.325777,0.784995,0.000000,0.0,0.499573
2002Q1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,7334.92,0.00,...,0.000000,0.906874,1.000000,0.389933,0.454545,0.415001,-1.000000,0.000022,0.0,0.804250


In [19]:
# Adding Global effective tax rate percentile
feature = '4Delta_abs_deduc_to_income_ratio'
new_name = '4D_deduc2income_perc_glob'
add_percentile(df=q_industries, col=feature, new_name=new_name)
q_industries.head()

,naics2,business_id,bus_loc_id,naics_code,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,sumsum_deduc,...,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,1Delta_abs_eff_tax_rate%,1Delta_abs_deduc_to_income_ratio,4Delta_abs_eff_tax_rate%,4Delta_abs_deduc_to_income_ratio,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob
date,,,,,,,,,,,,,,,,,,,,,
2001Q1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,16193.58,0.00,...,0.721733,0.909091,0.390064,0.454545,0.000000,0.000000,0.000000,0.0,0.500045,0.500000
2001Q2,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,158968.92,0.00,...,0.814791,1.000000,0.389619,0.454545,0.000020,0.000000,0.000000,0.0,0.500184,0.499954
2001Q3,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,95335.05,20497.55,...,0.625852,0.909091,0.912581,0.909091,-0.089222,0.215005,0.000000,0.0,0.500093,0.500047
2001Q4,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,36138.27,36138.27,...,0.003649,0.090909,0.997867,1.000000,-0.325777,0.784995,0.000000,0.0,0.499573,0.500190
2002Q1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,7334.92,0.00,...,0.906874,1.000000,0.389933,0.454545,0.415001,-1.000000,0.000022,0.0,0.804250,0.498629


In [20]:
# Add single industry percentile - 4 Delta Effective Tax Rate %
feature = '4Delta_abs_eff_tax_rate%'
new_name = '4D_eff_tax_perc_ind'


for naics2_code in q_industries['naics2'].unique():
    # Create a subset DataFrame for the current naics2 code
    single_industry_df = q_industries[q_industries['naics2'] == naics2_code].copy()
    
    # Calculate the percentiles within the group
    # (Obs: When applying the function, the assingment using loc did not work, so reconstruct calculation.)
    single_industry_df[new_name] = single_industry_df[[feature]].groupby(
                                            by=[single_industry_df.index.year, 
                                                single_industry_df.index.quarter
                                               ]).rank(method="average", pct=True)
 
    # Update the original DataFrame with the calculated percentiles
    q_industries.loc[q_industries['naics2'] == naics2_code, new_name] = single_industry_df[new_name]
    #print(q_industries[new_name].isnull().sum())

q_industries.head()

,naics2,business_id,bus_loc_id,naics_code,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,sumsum_deduc,...,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,1Delta_abs_eff_tax_rate%,1Delta_abs_deduc_to_income_ratio,4Delta_abs_eff_tax_rate%,4Delta_abs_deduc_to_income_ratio,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind
date,,,,,,,,,,,,,,,,,,,,,
2001Q1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,16193.58,0.00,...,0.909091,0.390064,0.454545,0.000000,0.000000,0.000000,0.0,0.500045,0.500000,0.545455
2001Q2,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,158968.92,0.00,...,1.000000,0.389619,0.454545,0.000020,0.000000,0.000000,0.0,0.500184,0.499954,0.545455
2001Q3,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,95335.05,20497.55,...,0.909091,0.912581,0.909091,-0.089222,0.215005,0.000000,0.0,0.500093,0.500047,0.545455
2001Q4,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,36138.27,36138.27,...,0.090909,0.997867,1.000000,-0.325777,0.784995,0.000000,0.0,0.499573,0.500190,0.545455
2002Q1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,7334.92,0.00,...,1.000000,0.389933,0.454545,0.415001,-1.000000,0.000022,0.0,0.804250,0.498629,0.545455


In [21]:
# Add single industry percentile - deduc to income ratio
# IMPORTANT: columns of the q_industries may not have to contain the percentiles
feature = '4Delta_abs_deduc_to_income_ratio'
new_name = '4D_deduc2income_perc_ind'
    
for naics2_code in q_industries['naics2'].unique():
    # Create a subset DataFrame for the current naics2 code
    single_industry_df = q_industries[q_industries['naics2'] == naics2_code].copy()
    
    # Calculate the percentiles (Obs: When applying the function, the assingment using loc where not working)
    single_industry_df[new_name] = single_industry_df[[feature]].groupby(
                                            by=[single_industry_df.index.year, 
                                                single_industry_df.index.quarter
                                               ]).rank(method="average", pct=True)
 
    # Update the original DataFrame with the calculated percentiles
    q_industries.loc[q_industries['naics2'] == naics2_code, new_name] = single_industry_df[new_name]
    #print(q_industries[new_name].isnull().sum())

q_industries.head()

,naics2,business_id,bus_loc_id,naics_code,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,sumsum_deduc,...,deduc2income_perc_glob,deduc2income_perc_ind,1Delta_abs_eff_tax_rate%,1Delta_abs_deduc_to_income_ratio,4Delta_abs_eff_tax_rate%,4Delta_abs_deduc_to_income_ratio,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind
date,,,,,,,,,,,,,,,,,,,,,
2001Q1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,16193.58,0.00,...,0.390064,0.454545,0.000000,0.000000,0.000000,0.0,0.500045,0.500000,0.545455,0.545455
2001Q2,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,158968.92,0.00,...,0.389619,0.454545,0.000020,0.000000,0.000000,0.0,0.500184,0.499954,0.545455,0.545455
2001Q3,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,95335.05,20497.55,...,0.912581,0.909091,-0.089222,0.215005,0.000000,0.0,0.500093,0.500047,0.545455,0.545455
2001Q4,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,36138.27,36138.27,...,0.997867,1.000000,-0.325777,0.784995,0.000000,0.0,0.499573,0.500190,0.545455,0.545455
2002Q1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,7334.92,0.00,...,0.389933,0.454545,0.415001,-1.000000,0.000022,0.0,0.804250,0.498629,0.545455,0.590909


In [22]:
q_industries.columns

Index(['naics2', 'business_id', 'bus_loc_id', 'naics_code',
       'business_legal_name', 'maxNumLoc', 'tax_period_cd', 'obl_type_id',
       'sumsum_gross', 'sumsum_deduc', 'sumsum_taxable', 'sumsum_paid',
       'eff_tax_rate%', 'deduc_to_income_ratio', 'eff_tax%_perc_glob',
       'eff_tax%_perc_ind', 'deduc2income_perc_glob', 'deduc2income_perc_ind',
       '1Delta_abs_eff_tax_rate%', '1Delta_abs_deduc_to_income_ratio',
       '4Delta_abs_eff_tax_rate%', '4Delta_abs_deduc_to_income_ratio',
       '4D_eff_tax%_perc_glob', '4D_deduc2income_perc_glob',
       '4D_eff_tax_perc_ind', '4D_deduc2income_perc_ind'],
      dtype='object')

In [23]:
q_industries.drop(columns=['1Delta_abs_eff_tax_rate%', '1Delta_abs_deduc_to_income_ratio'], inplace=True)
q_industries.columns

Index(['naics2', 'business_id', 'bus_loc_id', 'naics_code',
       'business_legal_name', 'maxNumLoc', 'tax_period_cd', 'obl_type_id',
       'sumsum_gross', 'sumsum_deduc', 'sumsum_taxable', 'sumsum_paid',
       'eff_tax_rate%', 'deduc_to_income_ratio', 'eff_tax%_perc_glob',
       'eff_tax%_perc_ind', 'deduc2income_perc_glob', 'deduc2income_perc_ind',
       '4Delta_abs_eff_tax_rate%', '4Delta_abs_deduc_to_income_ratio',
       '4D_eff_tax%_perc_glob', '4D_deduc2income_perc_glob',
       '4D_eff_tax_perc_ind', '4D_deduc2income_perc_ind'],
      dtype='object')

# Add Pecentiles on sales (size)

In [188]:
# Adding Global effective tax rate percentile
feature = 'sumsum_gross'
new_name = 'sumsum_gross_perc_glob'
add_percentile(df=q_industries, col=feature, new_name=new_name)
q_industries.head()

,index,naics2,business_id,bus_loc_id,naics_code,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,...,deduc2income_perc_ind,4Delta_abs_eff_tax_rate%,4Delta_abs_deduc_to_income_ratio,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind,k-cluster_ind,k-cluster_g,sumsum_gross_perc_glob
date,,,,,,,,,,,,,,,,,,,,,
2001Q1,0,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,16193.58,...,0.454545,0.000000,0.0,0.500045,0.500000,0.545455,0.545455,0.0,1,0.085914
2001Q2,1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,158968.92,...,0.454545,0.000000,0.0,0.500184,0.499954,0.545455,0.545455,0.0,1,0.586863
2001Q3,2,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,95335.05,...,0.909091,0.000000,0.0,0.500093,0.500047,0.545455,0.545455,0.0,1,0.467638
2001Q4,3,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,36138.27,...,1.000000,0.000000,0.0,0.499573,0.500190,0.545455,0.545455,1.0,2,0.241137
2002Q1,4,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,7334.92,...,0.454545,0.000022,0.0,0.804250,0.498629,0.545455,0.590909,0.0,1,0.023600


In [189]:
# Add single industry percentile - Effecctive Tax Rate %
# IMPORTANT: columns of the q_industries may not have to contain the percentiles
feature = 'sumsum_gross'
new_name = 'sumsum_gross_perc_ind'


for naics2_code in q_industries['naics2'].unique():
    # Create a subset DataFrame for the current naics2 code
    single_industry_df = q_industries[q_industries['naics2'] == naics2_code].copy()
    
    # Calculate the percentiles within the group
    # (Obs: When applying the function, the assingment using loc did not work, so reconstruct calculation.)
    single_industry_df[new_name] = single_industry_df[[feature]].groupby(
                                            by=[single_industry_df.index.year, 
                                                single_industry_df.index.quarter
                                               ]).rank(method="average", pct=True)
 
    # Update the original DataFrame with the calculated percentiles
    q_industries.loc[q_industries['naics2'] == naics2_code, new_name] = single_industry_df[new_name]
    #print(q_industries[new_name].isnull().sum())

q_industries.head()


,index,naics2,business_id,bus_loc_id,naics_code,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,...,4Delta_abs_eff_tax_rate%,4Delta_abs_deduc_to_income_ratio,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind,k-cluster_ind,k-cluster_g,sumsum_gross_perc_glob,sumsum_gross_perc_ind
date,,,,,,,,,,,,,,,,,,,,,
2001Q1,0,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,16193.58,...,0.000000,0.0,0.500045,0.500000,0.545455,0.545455,0.0,1,0.085914,0.090909
2001Q2,1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,158968.92,...,0.000000,0.0,0.500184,0.499954,0.545455,0.545455,0.0,1,0.586863,0.636364
2001Q3,2,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,95335.05,...,0.000000,0.0,0.500093,0.500047,0.545455,0.545455,0.0,1,0.467638,0.545455
2001Q4,3,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,36138.27,...,0.000000,0.0,0.499573,0.500190,0.545455,0.545455,1.0,2,0.241137,0.181818
2002Q1,4,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,7334.92,...,0.000022,0.0,0.804250,0.498629,0.545455,0.590909,0.0,1,0.023600,0.090909


# Clusters addition

Here we add the global clusters as well as the Industry level clusters

In [30]:
get_inertia_vs_k(X_final=q_industries, columns=['eff_tax%_perc_glob',
       'eff_tax%_perc_ind', 'deduc2income_perc_glob', 'deduc2income_perc_ind',
       '4D_eff_tax%_perc_glob', '4D_deduc2income_perc_glob',
       '4D_eff_tax_perc_ind', '4D_deduc2income_perc_ind']  , random_state = 19, plots=True)

:Curve   [k]   (inertia)

,k,inertia,inertia_fraction_with_respect_to_one_cluster,inertia_rate_of_decrease
0,1,6.777960e+06,1.000000,NaN
1,2,4.600647e+06,0.678766,-0.321234
2,3,3.095678e+06,0.456727,-0.222039
3,4,2.232066e+06,0.329312,-0.127415
4,5,1.971766e+06,0.290908,-0.038404
5,6,1.762905e+06,0.260094,-0.030815
6,7,1.536875e+06,0.226746,-0.033348
7,8,1.429236e+06,0.210865,-0.015881
8,9,1.337227e+06,0.197290,-0.013575
9,10,1.145775e+06,0.169044,-0.028246


## Global Clusters

In [31]:
# Adding the global cluster 'k-cluster_g'
q_industries, _, _ = get_kmean_clusters(X_final = q_industries, k=4, debug=True, random_state=19, 
                       columns=['eff_tax%_perc_glob',
                               'eff_tax%_perc_ind', 'deduc2income_perc_glob', 'deduc2income_perc_ind',
                               '4D_eff_tax%_perc_glob', '4D_deduc2income_perc_glob',
                               '4D_eff_tax_perc_ind', '4D_deduc2income_perc_ind'],
                       ref_cols=['naics2','business_id','bus_loc_id'],
                       cluster_col_name='k-cluster_g',       
                       return_only_ref_cols_and_cluster = False
                                    
)
q_industries.head()

Cluster value counts: 
 0    349605
1    328305
2     86636
3     82699
Name: k-cluster, dtype: int64

            Medians  


,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind
k-cluster,,,,,,,,
0,0.393112,0.424490,0.402359,0.417048,0.321755,0.499949,0.323077,0.500000
1,0.741592,0.751479,0.404305,0.420197,0.711328,0.499794,0.709951,0.500000
2,0.114246,0.156838,0.886854,0.877715,0.882817,0.063345,0.875083,0.071522
3,0.092330,0.121827,0.910233,0.907546,0.093766,0.947352,0.096944,0.944889



        Min Values 


,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind
k-cluster,,,,,,,,
0,0.000922,0.000559,0.337838,0.294529,0.000097,0.000193,0.000315,0.000332
1,0.032168,0.011792,0.337838,0.294529,0.076923,0.000102,0.165256,0.000328
2,0.000922,0.000403,0.383080,0.294529,0.000095,0.000091,0.000582,0.000315
3,0.000922,0.000351,0.675676,0.592549,0.000091,0.487797,0.000328,0.474576



        Max Values 


,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind
k-cluster,,,,,,,,
0,1.000000,1.000000,0.959245,0.974684,0.865734,0.576923,0.871972,0.875000
1,1.000000,1.000000,0.997193,1.000000,1.000000,1.000000,1.000000,1.000000
2,0.999907,1.000000,1.000000,1.000000,1.000000,0.515192,1.000000,1.000000
3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


,index,date,naics2,business_id,bus_loc_id,naics_code,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,...,deduc2income_perc_ind,4Delta_abs_eff_tax_rate%,4Delta_abs_deduc_to_income_ratio,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind,k-cluster,k-cluster,k-cluster
0,0,2001Q1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,...,0.454545,0.000000,0.0,0.500045,0.500000,0.545455,0.545455,1,1,1
1,1,2001Q2,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,...,0.454545,0.000000,0.0,0.500184,0.499954,0.545455,0.545455,1,1,1
2,2,2001Q3,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,...,0.909091,0.000000,0.0,0.500093,0.500047,0.545455,0.545455,1,1,1
3,3,2001Q4,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,...,1.000000,0.000000,0.0,0.499573,0.500190,0.545455,0.545455,2,2,2
4,4,2002Q1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,...,0.454545,0.000022,0.0,0.804250,0.498629,0.545455,0.590909,1,1,1


In [32]:
q_industries.set_index('date', inplace=True)

In [34]:
q_industries.head()

,index,naics2,business_id,bus_loc_id,naics_code,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,...,deduc2income_perc_ind,4Delta_abs_eff_tax_rate%,4Delta_abs_deduc_to_income_ratio,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind,k-cluster,k-cluster,k-cluster
date,,,,,,,,,,,,,,,,,,,,,
2001Q1,0,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,16193.58,...,0.454545,0.000000,0.0,0.500045,0.500000,0.545455,0.545455,1,1,1
2001Q2,1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,158968.92,...,0.454545,0.000000,0.0,0.500184,0.499954,0.545455,0.545455,1,1,1
2001Q3,2,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,95335.05,...,0.909091,0.000000,0.0,0.500093,0.500047,0.545455,0.545455,1,1,1
2001Q4,3,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,36138.27,...,1.000000,0.000000,0.0,0.499573,0.500190,0.545455,0.545455,2,2,2
2002Q1,4,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,7334.92,...,0.454545,0.000022,0.0,0.804250,0.498629,0.545455,0.590909,1,1,1


## industry clusters
### Analysis

In [35]:
q_industries, _, _ = get_kmean_clusters(X_final = q_industries, k=4, debug=True, random_state=19, 
                       columns=['eff_tax%_perc_glob',
                               'eff_tax%_perc_ind', 'deduc2income_perc_glob', 'deduc2income_perc_ind',
                               '4D_eff_tax%_perc_glob', '4D_deduc2income_perc_glob',
                               '4D_eff_tax_perc_ind', '4D_deduc2income_perc_ind'],
                       ref_cols=['naics2','business_id','bus_loc_id'],
                    return_only_ref_cols_and_cluster = False
                                    
)
q_industries.head()

Cluster value counts: 
 0    349605
1    328305
2     86636
3     82699
Name: k-cluster, dtype: int64

            Medians  


,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind
k-cluster,,,,,,,,
0,0.393112,0.424490,0.402359,0.417048,0.321755,0.499949,0.323077,0.500000
1,0.741592,0.751479,0.404305,0.420197,0.711328,0.499794,0.709951,0.500000
2,0.114246,0.156838,0.886854,0.877715,0.882817,0.063345,0.875083,0.071522
3,0.092330,0.121827,0.910233,0.907546,0.093766,0.947352,0.096944,0.944889



        Min Values 


,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind
k-cluster,,,,,,,,
0,0.000922,0.000559,0.337838,0.294529,0.000097,0.000193,0.000315,0.000332
1,0.032168,0.011792,0.337838,0.294529,0.076923,0.000102,0.165256,0.000328
2,0.000922,0.000403,0.383080,0.294529,0.000095,0.000091,0.000582,0.000315
3,0.000922,0.000351,0.675676,0.592549,0.000091,0.487797,0.000328,0.474576



        Max Values 


,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind
k-cluster,,,,,,,,
0,1.000000,1.000000,0.959245,0.974684,0.865734,0.576923,0.871972,0.875000
1,1.000000,1.000000,0.997193,1.000000,1.000000,1.000000,1.000000,1.000000
2,0.999907,1.000000,1.000000,1.000000,1.000000,0.515192,1.000000,1.000000
3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


,date,index,naics2,business_id,bus_loc_id,naics_code,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,...,4Delta_abs_eff_tax_rate%,4Delta_abs_deduc_to_income_ratio,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind,k-cluster,k-cluster,k-cluster,k-cluster
0,2001Q1,0,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,...,0.000000,0.0,0.500045,0.500000,0.545455,0.545455,1,1,1,1
1,2001Q2,1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,...,0.000000,0.0,0.500184,0.499954,0.545455,0.545455,1,1,1,1
2,2001Q3,2,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,...,0.000000,0.0,0.500093,0.500047,0.545455,0.545455,1,1,1,1
3,2001Q4,3,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,...,0.000000,0.0,0.499573,0.500190,0.545455,0.545455,2,2,2,2
4,2002Q1,4,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,...,0.000022,0.0,0.804250,0.498629,0.545455,0.590909,1,1,1,1


In [38]:
# REASON WHY WE WILL NOT USE SAME PERCENTILES FOR INDUSTRY THAN FOR GLOBAL (at least at MVP)
for naics2_code in q_industries['naics2'].unique():
    print(f"Analizing industry: {naics2}")
    
    # Calculate the clusters in each group
    single_industry_clusters_df, _, _ = get_kmean_clusters(X_final = q_industries[q_industries['naics2'] == naics2_code], 
                                                    k=4, debug=True, random_state=19, 
                                                   columns=['eff_tax%_perc_glob',
                                                           'eff_tax%_perc_ind', 'deduc2income_perc_glob', 'deduc2income_perc_ind',
                                                            '4D_eff_tax%_perc_glob', '4D_deduc2income_perc_glob',
                                                            '4D_eff_tax_perc_ind', '4D_deduc2income_perc_ind'],
                                                   ref_cols=['naics2','business_id','bus_loc_id'],
                                                   cluster_col_name = 'k-cluster_ind',
                                                   return_only_ref_cols_and_cluster = True
                                    
)
# OBS: There appear two compliant clusters inside the industries

Cluster value counts: 
 0    430
1    383
2    129
3     94
Name: k-cluster_ind, dtype: int64

            Medians  


,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind
k-cluster_ind,,,,,,,,
0,0.294467,0.486842,0.407904,0.443750,0.260323,0.499535,0.375000,0.555556
1,0.559657,0.777778,0.407599,0.437500,0.781321,0.499949,0.777778,0.545455
2,0.024395,0.142857,0.978695,0.937500,0.086755,0.952130,0.187500,0.937500
3,0.058042,0.214286,0.952945,0.897368,0.936776,0.037537,0.888889,0.114379



        Min Values 


,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind
k-cluster_ind,,,,,,,,
0,0.002546,0.083333,0.383080,0.343750,0.001395,0.003223,0.050000,0.083333
1,0.166716,0.250000,0.383080,0.343750,0.282319,0.000357,0.333333,0.052632
2,0.002239,0.050000,0.795404,0.687500,0.001343,0.499954,0.050000,0.545455
3,0.002819,0.050000,0.386588,0.400000,0.028217,0.000215,0.071429,0.050000



        Max Values 


...

,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind
k-cluster_ind,,,,,,,,
0,0.405304,0.328830,0.392461,0.414530,0.286901,0.499954,0.314695,0.500582
1,0.748989,0.765655,0.392461,0.414530,0.736576,0.499949,0.698238,0.500577
2,0.146770,0.107056,0.865530,0.899808,0.877756,0.066080,0.861320,0.055357
3,0.118677,0.082459,0.891013,0.924148,0.102035,0.943402,0.110764,0.955433



        Min Values 


,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind
k-cluster_ind,,,,,,,,
0,0.000922,0.001036,0.337838,0.401320,0.000098,0.000453,0.000575,0.000637
1,0.115248,0.086728,0.337838,0.401320,0.102441,0.000154,0.104796,0.000575
2,0.000962,0.001036,0.383080,0.401320,0.000095,0.000095,0.000582,0.000577
3,0.001019,0.001385,0.739130,0.804797,0.000104,0.495191,0.000577,0.495690



        Max Values 


,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind
k-cluster_ind,,,,,,,,
0,1.000000,1.000000,0.851030,0.888821,0.953430,0.515192,0.956820,0.545455
1,1.000000,1.000000,0.983998,1.000000,1.000000,0.958141,1.000000,1.000000
2,0.999907,1.000000,0.999659,1.000000,1.000000,0.515192,1.000000,0.545455
3,0.999905,1.000000,0.999704,1.000000,1.000000,1.000000,1.000000,1.000000


In [63]:
#delete
q_industries.index, q_industries.shape

(RangeIndex(start=0, stop=847245, step=1), (847245, 31))

### Adding Internal Industry Clusters

    Here we add the cluster for each industry in the column 'k-cluster_ind'

In [92]:
#Adding internal industry clusters
cluster_col_name='k-cluster_ind'
for naics2_code in q_industries['naics2'].unique():
    print("")
    print(f" Analisis naics: {naics2_code}")
    get_inertia_vs_k(X_final=q_industries[q_industries['naics2']==naics2_code], 
                     columns=['eff_tax_rate%',
                            'deduc_to_income_ratio',
                            '4Delta_abs_eff_tax_rate%', '4Delta_abs_deduc_to_income_ratio'
                             ],
                      random_state = 19, plots=True)

    single_ind_clusters_df = pd.DataFrame()
    single_ind_clusters_df, _, _ = get_kmean_clusters(X_final = q_industries[q_industries['naics2'] == naics2_code], 
                                                        k=4, debug=True, random_state=19, 
                                                       columns=['eff_tax_rate%',
                                                                'deduc_to_income_ratio',
                                                                '4Delta_abs_eff_tax_rate%', '4Delta_abs_deduc_to_income_ratio'
                                                               ],
                                                       ref_cols=['naics2','business_id','bus_loc_id'],
                                                       cluster_col_name=cluster_col_name,
                                                       return_only_ref_cols_and_cluster = True
    )
    # Make sure is int to not have trouble in combinations
    single_ind_clusters_df[cluster_col_name] = single_ind_clusters_df[cluster_col_name].astype(float)
    display(single_ind_clusters_df.head())
    print(single_ind_clusters_df.isnull().sum())
 
    # Update the original DataFrame with the calculated percentiles
    q_industries.loc[q_industries['naics2'] == naics2_code, cluster_col_name] = single_ind_clusters_df[cluster_col_name].values
    #q_industries[cluster_col_name] = q_industries[cluster_col_name].astype(int)
    print("")
    print('single_ind_clusters_df[cluster_col_name] :')
    display(single_ind_clusters_df[cluster_col_name])
    print(single_ind_clusters_df[cluster_col_name].size)
    print("")
    print("q_industries.loc[q_industries['naics2'] == naics2_code, cluster_col_name] :")
    display(q_industries.loc[q_industries['naics2'] == naics2_code, cluster_col_name])
    display(q_industries.loc[q_industries['naics2'] == naics2_code, cluster_col_name].size)




 Analisis naics: 22


:Curve   [k]   (inertia)

Cluster value counts: 
 0    867
1    132
2     33
3      4
Name: k-cluster_ind, dtype: int64

            Medians  


,eff_tax_rate%,deduc_to_income_ratio,4Delta_abs_eff_tax_rate%,4Delta_abs_deduc_to_income_ratio
k-cluster_ind,,,,
0,0.221981,0.000000,0.000000,0.000000
1,0.017420,0.935535,0.000000,0.000000
2,0.076143,0.828841,-0.181490,0.807941
3,11.225385,0.000000,10.837050,0.000000



        Min Values 


...

naics2           0
business_id      0
bus_loc_id       0
k-cluster_ind    0
dtype: int64

single_ind_clusters_df[cluster_col_name] :


0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
131694    0.0
131695    0.0
131696    0.0
131697    0.0
131698    0.0
Name: k-cluster_ind, Length: 131699, dtype: float64

131699

q_industries.loc[q_industries['naics2'] == naics2_code, cluster_col_name] :


715546    0.0
715547    0.0
715548    0.0
715549    0.0
715550    0.0
         ... 
847240    0.0
847241    0.0
847242    0.0
847243    0.0
847244    0.0
Name: k-cluster_ind, Length: 131699, dtype: float64

131699

In [73]:
# Comparing clustering features
cluster_col_name='k-cluster_ind'
naics2_code=='22'
print("")
print(f" Analisis naics: {naics2_code}")
get_inertia_vs_k(X_final=q_industries[q_industries['naics2']==naics2_code], 
                 columns=['eff_tax_rate%',
                        'deduc_to_income_ratio',
                        '4Delta_abs_eff_tax_rate%', '4Delta_abs_deduc_to_income_ratio'
                         ],
                  random_state = 19, plots=True)

single_ind_clusters_df = pd.DataFrame()
single_ind_clusters_df, _, _ = get_kmean_clusters(X_final = q_industries[q_industries['naics2'] == naics2_code], 
                                                    k=4, debug=True, random_state=19, 
                                                   columns=['eff_tax_rate%',
                                                            'deduc_to_income_ratio',
                                                            '4Delta_abs_eff_tax_rate%', '4Delta_abs_deduc_to_income_ratio'
                                                           ],
                                                   ref_cols=['naics2','business_id','bus_loc_id'],
                                                   cluster_col_name=cluster_col_name,
                                                   return_only_ref_cols_and_cluster = True
)
# Make sure is int to not have trouble in combinations
single_ind_clusters_df[cluster_col_name] = single_ind_clusters_df[cluster_col_name].astype(int)
display(single_ind_clusters_df.head())
print(single_ind_clusters_df.isnull().sum())

# Update the original DataFrame with the calculated percentiles
q_industries.loc[q_industries['naics2'] == naics2_code, cluster_col_name] = single_ind_clusters_df[cluster_col_name].astype(int)
#q_industries[cluster_col_name] = q_industries[cluster_col_name].astype(int)
print("")
print('single_ind_clusters_df[cluster_col_name] :')
display(single_ind_clusters_df[cluster_col_name])
print(single_ind_clusters_df[cluster_col_name].size)
print("")
print("q_industries.loc[q_industries['naics2'] == naics2_code, cluster_col_name] :")
display(q_industries.loc[q_industries['naics2'] == naics2_code, cluster_col_name])
display(q_industries.loc[q_industries['naics2'] == naics2_code, cluster_col_name].size)




 Analisis naics: 22


:Curve   [k]   (inertia)

Cluster value counts: 
 0    867
1    132
2     33
3      4
Name: k-cluster_ind, dtype: int64

            Medians  


,eff_tax_rate%,deduc_to_income_ratio,4Delta_abs_eff_tax_rate%,4Delta_abs_deduc_to_income_ratio
k-cluster_ind,,,,
0,0.221981,0.000000,0.000000,0.000000
1,0.017420,0.935535,0.000000,0.000000
2,0.076143,0.828841,-0.181490,0.807941
3,11.225385,0.000000,10.837050,0.000000



        Min Values 


...

naics2           0
business_id      0
bus_loc_id       0
k-cluster_ind    0
dtype: int64

single_ind_clusters_df[cluster_col_name] :


0       0
1       0
2       0
3       1
4       0
       ..
1031    0
1032    0
1033    0
1034    0
1035    0
Name: k-cluster_ind, Length: 1036, dtype: int64

1036

q_industries.loc[q_industries['naics2'] == naics2_code, cluster_col_name] :


0       0.0
1       0.0
2       0.0
3       1.0
4       0.0
       ... 
1031    0.0
1032    0.0
1033    0.0
1034    0.0
1035    0.0
Name: k-cluster_ind, Length: 1036, dtype: float64

1036

In [167]:
# Checking on the order of clusters
# Also looking at the sizes of the industries

large_industries = ['23','54','81', '42', '45']
medium_industries = ['56', '53', '33', '61']
small_industries = [q for q in q_industries['naics2'].unique() if q not in large_industries + medium_industries]

plot1 = q_industries.loc[q_industries['naics2'].isin(large_industries)].groupby(['naics2', 'k-cluster_ind']).size().hvplot(
            kind='bar',
            title="Large Industry Clusters",
            height=300,
            width=800,
            xlabel="NAICS and Industry Cluster Number",
            ylabel="Amount of Periods of Sale Taxes"
)

plot2 = q_industries.loc[q_industries['naics2'].isin(medium_industries)].groupby(['naics2', 'k-cluster_ind']).size().hvplot(
            kind='bar',
            title="Large Industry Clusters",
            height=300,
            width=800,
            xlabel="NAICS and Industry Cluster Number",
            ylabel="Amount of Periods of Sale Taxes"
)

#max_value = q_industries[q_industries['naics2'].isin(small_industries)].groupby('naics2')['k-cluster_ind'].count().max()
plot3 = q_industries.loc[q_industries['naics2'].isin(small_industries)].groupby(['naics2', 'k-cluster_ind']).size().hvplot(
            kind='bar',
            title="Small Industry Clusters",
            height=300,
            width=800,
            xlabel="NAICS and Industry Cluster Number",
            ylabel="Amount of Periods of Sale Taxes"
)


display(plot1)
display(plot2)
display(plot3)

:Bars   [naics2,k-cluster_ind]   (0)

:Bars   [naics2,k-cluster_ind]   (0)

:Bars   [naics2,k-cluster_ind]   (0)

In [ ]:
# Relabel k_cluster_ind and global for 'is_compliant_cluster'
q_industries['is_ccluster_ind']=np.where(q_industries['k-cluster_ind']==0,1,0)
q_industries['is_ccluster_g']=np.where(q_industries['k-cluster_g'].isin([0,1]),1,0)
q_industries[q_industries['k-cluster_ind']!=0][['k-cluster_g','is_ccluster_g','k-cluster_ind','is_ccluster_ind']]

%store q_industries

In [6]:
%store -r q_industries
q_industries.head()

,index,naics2,business_id,bus_loc_id,naics_code,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,...,4D_eff_tax%_perc_glob,4D_deduc2income_perc_glob,4D_eff_tax_perc_ind,4D_deduc2income_perc_ind,k-cluster_ind,k-cluster_g,sumsum_gross_perc_glob,sumsum_gross_perc_ind,is_ccluster_ind,is_ccluster_g
date,,,,,,,,,,,,,,,,,,,,,
2001Q1,0,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,16193.58,...,0.500045,0.500000,0.545455,0.545455,0.0,1,0.085914,0.090909,1,1
2001Q2,1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,158968.92,...,0.500184,0.499954,0.545455,0.545455,0.0,1,0.586863,0.636364,1,1
2001Q3,2,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,95335.05,...,0.500093,0.500047,0.545455,0.545455,0.0,1,0.467638,0.545455,1,1
2001Q4,3,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,36138.27,...,0.499573,0.500190,0.545455,0.545455,1.0,2,0.241137,0.181818,0,0
2002Q1,4,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,7334.92,...,0.804250,0.498629,0.545455,0.590909,0.0,1,0.023600,0.090909,1,1


## Adding cluster counts

In [39]:
# Parameters
ref_cols=['naics2','business_id','bus_loc_id']

In [41]:
cluster_counts_df = get_cluster_counts(
        data_clustered = q_industries,
        cluster_col = 'k-cluster_ind',
        compliant_cluster=0,
        ref_cols=ref_cols,
        extend_data_clustered = True,
        include_out_of_compliance_column=True,
        sort_values=False)
cluster_counts_df



k-cluster_ind,naics2,business_id,bus_loc_id,0.0,1.0,2.0,3.0,non_compliant_cluster,compliant_cluster,prop_outside_compliant_cluster
0,22,109874,110874,7,1,0,0,1,7,0.125000
1,22,129983,131500,7,0,0,0,0,7,0.000000
2,22,131014,132555,38,41,9,0,50,38,0.568182
3,22,179077,181638,11,4,0,0,4,11,0.266667
4,22,185610,188153,0,1,0,0,1,0,1.000000
...,...,...,...,...,...,...,...,...,...,...
28457,81,98622,99232,12,0,0,0,0,12,0.000000
28458,81,98810,99430,23,0,0,0,0,23,0.000000
28459,81,99641,100278,86,0,0,0,0,86,0.000000
28460,81,99677,100315,43,0,0,0,0,43,0.000000


In [33]:
q_industries

,index,naics2,business_id,bus_loc_id,naics_code,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,...,k-cluster_ind,k-cluster_g,sumsum_gross_perc_glob,sumsum_gross_perc_ind,is_ccluster_ind,is_ccluster_g,max_outlier_score,hd-cluster,out_of_compliance,prop_outside_compliant_cluster
date,,,,,,,,,,,,,,,,,,,,,
2001Q1,0,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,16193.58,...,0.0,1,0.085914,0.090909,1,1,0.711264,1.0,0,0.125
2001Q2,1,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,158968.92,...,0.0,1,0.586863,0.636364,1,1,0.711264,1.0,0,0.125
2001Q3,2,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,95335.05,...,0.0,1,0.467638,0.545455,1,1,0.711264,0.0,0,0.125
2001Q4,3,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,36138.27,...,1.0,2,0.241137,0.181818,0,0,0.711264,0.0,1,0.125
2002Q1,4,22,109874,110874,221122,ISLAND UTILITY COMPANY,1,Q,10,7334.92,...,0.0,1,0.023600,0.090909,1,1,0.711264,1.0,0,0.125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012Q4,847240,81,99932,100589,811430,RAMUTA RONALD K,1,Q,10,38792.45,...,0.0,0,0.073894,0.072281,1,1,NaN,NaN,0,0.000
2013Q1,847241,81,99932,100589,811430,RAMUTA RONALD K,1,Q,10,33359.00,...,0.0,0,0.063256,0.057163,1,1,NaN,NaN,0,0.000
2013Q2,847242,81,99932,100589,811430,RAMUTA RONALD K,1,Q,10,30774.00,...,0.0,0,0.043253,0.037853,1,1,NaN,NaN,0,0.000


# Adding outlier score features

## Including Features for Outlier Score

    Here we include features in relation to classifications of the points as outliers in internal clusters of the industries.
    
    We will include:
    1) max_outlier_score (from get_outlier_scores function)
    2) hd-cluster (from get_outlier_score function)
    3) proportion outside the outlier cluster (from get cluster counts function)

In [196]:
get_outlier_scores?

Signature:
get_outlier_scores(
    data_input: pandas.core.frame.DataFrame,
    set_of_features: List = None,
    ref_cols: List = ['date', 'naics_code', 'business_id', 'bus_loc_id'],
    min_cluster_size: Optional[int] = None,
    min_samples: Optional[int] = None,
    window_size_for_rolling: int = 4,
    add_outlier_score_to_data_input: bool = True,
    add_hd_cluster_column_to_data: bool = True,
    show_hdbscan_cluster_classification: bool = True,
) -> pandas.core.frame.DataFrame
Docstring:
Get outlier scores using HDBSCAN clustering algorithm for the given data.
This are GLOSH scores, which considers local density of the points.

Parameters:
    data_input (pd.DataFrame): The input DataFrame containing the data.
    set_of_features (List, optional): List of feature names to 
        consider. If None, all numeric columns are selected.
    min_cluster_size (int, optional): The minimum size of clusters.
        Defaults to 10% of the data, or 2, whatever is maximum.
    min_samples

In [28]:
# THIS CODE BREAKS DUE TO LACK OF MEMORY STARTING ON INDUSTRY NAICS2: 23
# Parameters
set_of_features = ['eff_tax_rate%',
                    'deduc_to_income_ratio',
                 #   '4Delta_abs_eff_tax_rate%', '4Delta_abs_deduc_to_income_ratio'
                   ]
ref_cols=['naics2','business_id','bus_loc_id']

data = q_industries[q_industries['naics2']=='42'][ref_cols + set_of_features].reset_index()
data.head()

# Global outlier scores will not be performed, since it broke whe I tried. Can be done in a stronger machine

# Addition of industry percentiles
for naics2_code in q_industries['naics2'].unique():
    print(f"working on cluster {naics2_code}")
    outlier_scores_df = pd.DataFrame()
    data_df = q_industries[q_industries['naics2']==naics2_code][ref_cols + set_of_features].reset_index()
    get_outlier_scores(data_input = data_df,
                               set_of_features=set_of_features,
                               ref_cols = ref_cols,
                               add_outlier_score_to_data_input = True,
                               add_hd_cluster_column_to_data = True,
                               show_hdbscan_cluster_classification = True
                                )
 
    all_col_names = ['outlier_score', 'rolling_outlier_score', 'max_outlier_score']
    added_col_names = ['max_outlier_score', 'hd-cluster']

    # Update the original DataFrame with the outlier scores
    q_industries.loc[q_industries['naics2'] == naics2_code, added_col_names] = data_df[added_col_names].values

    print("")
    print("q_industries.loc[q_industries['naics2'] == naics2_code, added_col_names] :")
    display(q_industries.loc[q_industries['naics2'] == naics2_code, added_col_names].size)
    display(q_industries.loc[q_industries['naics2'] == naics2_code, added_col_names])

working on cluster 22
HDBSCAN Cluster Classifications in Data:
 3    340
 1    244
 2    204
 0    172
-1     76
Name: hd-cluster, dtype: int64

q_industries.loc[q_industries['naics2'] == naics2_code, added_col_names] :


2072

,max_outlier_score,hd-cluster
date,,
2001Q1,0.711264,1.0
2001Q2,0.711264,1.0
2001Q3,0.711264,0.0
2001Q4,0.711264,0.0
2002Q1,0.711264,1.0
...,...,...
2021Q3,0.603821,2.0
2021Q4,0.603821,2.0
2022Q1,0.603821,2.0


working on cluster 23


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}